In [1]:
!pip install openai
import os
import openai
import pandas as pd
from openai import OpenAI

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd drive/MyDrive/llm_bias_expr

/content/drive/MyDrive/llm_bias_expr


In [4]:
!ls

cuisines.gdoc					   results_irr_tf_rel_long_dish_first.csv
dietary_restrictions.gdoc			   results_irr_tf_rel_long_dish_hypo.csv
irrlevant.gdoc					   results_irr_tf_rel_long_dish_third.csv
llm_bias_prompt_mc_rel_simp_cuisine.csv		   results_mc_rel_simp_cuisine.csv
llm_bias_prompt_mc_rel_simp_dish.csv		   results_mc_rel_simp_dish.csv
llm_bias_prompt_mc_rel_simp_dish.xlsx		   results_open_mixed_short_recipe_first.csv
llm_bias_prompt_open_mixed_short_recipe_first.csv  results_open_mixed_short_recipe_hypo.csv
llm_bias_prompt_open_mixed_short_recipe_hypo.csv   results_open_mixed_short_recipe_third.csv
llm_bias_prompt_open_mixed_short_recipe_third.csv  results_open_mixed_short_recipe_third.gsheet
llm_bias_prompt_open_rel_short_recipe.csv	   results_open_rel_short_recipe.csv
llm_bias_prompt_tf_rel_long_dish_first.csv	   results_tf_rel_long_dish_first.csv
llm_bias_prompt_tf_rel_long_dish_hypo.csv	   results_tf_rel_long_dish_hypo.csv
llm_bias_prompt_tf_rel_long_dish_third.csv	   results_tf_rel

In [14]:
prompts_df = pd.read_csv('llm_bias_prompt_tf_rel_long_dish_first.csv').dropna(axis=1)

DIET_PROMPTS = prompts_df['Prompt'].tolist()

In [15]:
prompts_df

,Type,Length,Content,Perspective,Answer Type,Prompt
0,Dietary Advice,Long,Relevant,First,True/False,I wonder if a serving of Greek Lemon Chicken w...
1,Dietary Advice,Long,Relevant,First,True/False,I wonder if a serving of Greek Lemon Chicken w...
2,Dietary Advice,Long,Relevant,First,True/False,I wonder if a serving of Greek Lemon Chicken w...
3,Dietary Advice,Long,Relevant,First,True/False,I wonder if a serving of Greek Lemon Chicken w...
4,Dietary Advice,Long,Relevant,First,True/False,I wonder if a serving of West African Chicken ...
...,...,...,...,...,...,...
235,Dietary Advice,Long,Relevant,First,True/False,I am a Native American non-binary. I wonder if...
236,Dietary Advice,Long,Relevant,First,True/False,I am a Native American non-binary. I wonder if...
237,Dietary Advice,Long,Relevant,First,True/False,I am a Native American non-binary. I wonder if...
238,Dietary Advice,Long,Relevant,First,True/False,I am a Native American non-binary. I wonder if...


In [16]:
def modify_prompts(df, restrictions):

    frames = []

    for restriction in restrictions:
        temp_df = df.copy()

        temp_df.loc[temp_df['Perspective'] == 'First', 'Prompt'] = \
            "I have " + restriction + ". " + temp_df['Prompt'].str.replace("I want to", "I want to", regex=False)

        temp_df.loc[temp_df['Perspective'] == 'Third', 'Prompt'] = \
            temp_df['Prompt'].str.replace(" wants to", " with " + restriction + " wants to", regex=False)

        temp_df.loc[temp_df['Perspective'] == 'Hypothetical', 'Prompt'] = \
            "Help my friend, who has " + restriction + ", " + temp_df['Prompt'].str.replace("Help ", "", regex=False)

        frames.append(temp_df)

    modified_df = pd.concat(frames, ignore_index=True)
    return modified_df


dietary_restrictions = ['lactose intolerance', 'gluten intolerance', 'vegetarian', 'diabetes', 'peanut allergy']
modified_prompts_df = modify_prompts(prompts_df, dietary_restrictions)
modified_prompts_df['Length'] = 'Long'
modified_prompts_df

,Type,Length,Content,Perspective,Answer Type,Prompt
0,Dietary Advice,Long,Relevant,First,True/False,I have lactose intolerance. I wonder if a serv...
1,Dietary Advice,Long,Relevant,First,True/False,I have lactose intolerance. I wonder if a serv...
2,Dietary Advice,Long,Relevant,First,True/False,I have lactose intolerance. I wonder if a serv...
3,Dietary Advice,Long,Relevant,First,True/False,I have lactose intolerance. I wonder if a serv...
4,Dietary Advice,Long,Relevant,First,True/False,I have lactose intolerance. I wonder if a serv...
...,...,...,...,...,...,...
1195,Dietary Advice,Long,Relevant,First,True/False,I have peanut allergy. I am a Native American ...
1196,Dietary Advice,Long,Relevant,First,True/False,I have peanut allergy. I am a Native American ...
1197,Dietary Advice,Long,Relevant,First,True/False,I have peanut allergy. I am a Native American ...
1198,Dietary Advice,Long,Relevant,First,True/False,I have peanut allergy. I am a Native American ...


In [17]:

OPENAI_API_KEY = 


client = OpenAI(api_key=OPENAI_API_KEY)

def chatWithGPT(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

def run_prompts(prompts, csv_file_path, num_runs=10):
    metadata = []
    for _, row in prompts_df.iterrows():
        for _ in range(num_runs):
            response = chatWithGPT(row['Prompt'])
            if response is not None:
                metadata.append({
                    "Type": row['Type'],
                    "Length": row['Length'],
                    "Content": row['Content'],
                    "Perspective": row['Perspective'],
                    "Answer Type": row['Answer Type'],
                    "Prompt": row['Prompt'],
                    "Response": response
                })

    df = pd.DataFrame(metadata)
    df.to_csv(csv_file_path, index=False)



In [18]:
result_path = "results_tf_rel_longer_dish_first.csv"
run_prompts(modified_prompts_df, result_path)